In [3]:
import streamlit_functions as sf
import importlib as imp
import geopandas as gpd
import matplotlib.pyplot as plt
import functions as f
# debugging
import importlib as imp
imp.reload(f)

from shapely.geometry import MultiLineString



In [2]:
bikelane_all = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk_crs_25832.parquet')

# filtering bikelanes which goes through a forest area

Read in

In [3]:
# ok we cant read in a 150 MB multilinestring for sure. Fuck
# bikelane_all_mp_wgs84 = gpd.read_parquet('dataset/raw_unprocessed/bikelane_wgs84_multilinestring.parquet')
bikelane_all_wgs84 = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk_WGS84_h3_indexed.parquet') 
forest_all_wgs84 = gpd.read_parquet('dataset/raw_unprocessed/denmark_landuse_forest_crs_wgs84.parquet')

In [ ]:
print(bikelane_all_wgs84.crs)
print(forest_all_wgs84.crs)
print(bikelane_all_wgs84.head())
print(forest_all_wgs84.head())
# print ther set of the geometry objects in both dataset
print("Set of geometry objects")
print(set(forest_all_wgs84.geom_type))
print(set(bikelane_all_wgs84.geom_type))

### MAssive data cleaning

minor filtering, keep only polygons and multipolygons

In [ ]:
forest_all_wgs84_only_polygons = forest_all_wgs84[forest_all_wgs84.geom_type.isin(['Polygon', 'MultiPolygon'])]
# forest_all_wgs84_only_polygons = forest_all_wgs84[forest_all_wgs84.geom_type.isin(['Polygon'])]

In [55]:
# first maybe it helps if I explode the multipolygons
forest_all_wgs84_only_polygons_exploded = forest_all_wgs84_only_polygons.explode()

C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_13524\4093333767.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  forest_all_wgs84_only_polygons_exploded = forest_all_wgs84_only_polygons.explode()


In [56]:
# and needs to be unary unionized
forest_all_wgs84_only_polygons_exploded_unary_union = forest_all_wgs84_only_polygons_exploded.unary_union

In [60]:
forest_all_wgs84_only_polygons_exploded_unary_union_df = gpd.GeoDataFrame(geometry=[forest_all_wgs84_only_polygons_exploded_unary_union])
forest_all_wgs84_only_polygons_exploded_unary_union_df  = df.explode().reset_index(drop=True)

C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_13524\1754656862.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  forest_all_wgs84_only_polygons_exploded_unary_union_df  = df.explode().reset_index(drop=True)


In [64]:
forest_all_wgs84_only_polygons_exploded_unary_union_df.crs = f.CRS
forest_all_wgs84_only_polygons_exploded_unary_union_df.to_parquet('dataset/processed/forest_areas_with_bikelanes_wgs84_TEMP.parquet')

### The sampling method is fucking wrong, make samples

In [4]:
# read in 
forest_all_wgs84_only_polygons_2 = gpd.read_parquet('dataset/processed/forest_areas_with_bikelanes_wgs84_TEMP.parquet')
bikelane_all_wgs84 = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk_WGS84_h3_indexed.parquet') 

In [5]:
F_sample = 10000
B_sample = 3000
forest = forest_all_wgs84_only_polygons_2.sample(F_sample)
#
# bikes = bikelane_all_wgs84.sample(B_sample)
bikes = bikelane_all_wgs84

In [6]:
print(bikes.crs)
print(forest.crs)
print(bikes.head())
print(forest.head())
# print ther set of the geometry objects in both dataset
print("Set of geometry objects")
print(set(forest.geom_type))
print(set(bikes.geom_type))

{"$schema": "https://proj.org/schemas/v0.7/projjson.schema.json", "type": "GeographicCRS", "name": "WGS 84", "datum": {"type": "GeodeticReferenceFrame", "name": "World Geodetic System 1984", "ellipsoid": {"name": "WGS 84", "semi_major_axis": 6378137, "inverse_flattening": 298.257223563}}, "coordinate_system": {"subtype": "ellipsoidal", "axis": [{"name": "Geodetic latitude", "abbreviation": "Lat", "direction": "north", "unit": "degree"}, {"name": "Geodetic longitude", "abbreviation": "Lon", "direction": "east", "unit": "degree"}]}, "scope": "Horizontal component of 3D system.", "area": "World.", "bbox": {"south_latitude": -90, "west_longitude": -180, "north_latitude": 90, "east_longitude": 180}, "id": {"authority": "EPSG", "code": 4326}}
{"$schema": "https://proj.org/schemas/v0.7/projjson.schema.json", "type": "GeographicCRS", "name": "WGS 84", "datum_ensemble": {"name": "World Geodetic System 1984 ensemble", "members": [{"name": "World Geodetic System 1984 (Transit)"}, {"name": "World 

### We have to create a multilinestring from the bikelane dataset

In [7]:
multilinestring_bike = MultiLineString(list(bikes.geometry))

In [8]:
multilinestring_bike

KeyboardInterrupt: 

#### The main logic to filter out areas

In [9]:
# Initialize the "keep" column with False for all rows
forest["keep"] = False

# Iterate through each polygon in df1 using itertuples
c = 0
print("START")
for row in forest.itertuples(index=True):
    print("First Run")
    if multilinestring_bike.intersects(row.geometry):
        # Set the "keep" value to True for intersecting polygons
        #forest.at[row.Index, "keep"] = True
        None
    c += 1
    print(c)
        
forest[forest["keep"] == True].explore()

START
First Run


KeyboardInterrupt: 

### Filtering method 1

In [ ]:
forest_areas_touched_or_crossed = gpd.overlay(
    bikelane_all_wgs84, forest_all_wgs84_only_polygons, 
    how='intersection', keep_geom_type=False
)

In [ ]:
forest_areas_touched_or_crossed.head()

In [ ]:
# Assuming gdf_denmark is your bike lanes (lines) and forest_areas_with_bikelanes is your forest areas (polygons)

# Perform the overlay with keep_geom_type=False to keep all intersecting geometries
###### PROBABLY THIS IS WRONG
forest_areas_touched_or_crossed = gpd.overlay(
    forest_all_wgs84_only_polygons, bikelane_all_wgs84, 
    how='intersection', keep_geom_type=False
)

# Display the head of the resulting GeoDataFrame
print(forest_areas_touched_or_crossed.head())

# If the above does not give the expected results (i.e., retaining polygons), try spatial join as an alternative
from geopandas.tools import sjoin

# Find all forest polygons that intersect (touch or cross) any bike lane
# Using spatial join to identify all polygons that intersect line geometries
forest_areas_intersected = sjoin(
    forest_all_wgs84_only_polygons, bikelane_all_wgs84, 
    how='inner', op='intersects'
)

# Display the head of the intersected forests
print(forest_areas_intersected.head())

check the details of the dataset

In [ ]:
imp.reload(f)
f.print_gdf_details(forest_areas_intersected)

#### Keep only the necessary columns in the dataset

In [ ]:
forest_areas_intersected_cleaned = forest_areas_intersected.drop_duplicates()
forest_areas_intersected_cleaned = forest_areas_intersected_cleaned["geometry"]
f.print_gdf_details(forest_areas_intersected_cleaned)

In [ ]:
forest_areas_intersected_cleaned.sample(5000).explore()

and save it 

In [ ]:
# Assuming bikelane_all_wgs84 and forest_all_wgs84 are already loaded as GeoDataFrames

# Step 1: Perform the spatial join
intersected_forests = gpd.sjoin(forest_all_wgs84, bikelane_all_wgs84, how="inner", op='intersects')

# Remove any duplicates based on geometries directly to ensure uniqueness
# Convert geometries to WKT string which is hashable and can be used to drop duplicates
intersected_forests['geometry_wkt'] = intersected_forests.geometry.apply(lambda x: x.wkt)
forest_areas_with_bikelanes = intersected_forests.drop_duplicates(subset='geometry_wkt')

# Remove the helper column and retain the original GeoDataFrame format
forest_areas_with_bikelanes = forest_areas_with_bikelanes.drop(columns=['geometry_wkt'])
forest_areas_with_bikelanes.drop(columns=['index_right0', 'index_right1', 'index_right2', 'h3_index'], inplace=True)

### Quick check

In [ ]:
print(set(forest_areas_with_bikelanes.geom_type))

# delete geom types which are not Polygon or MultiPolygon
forest_areas_with_bikelanes = forest_areas_with_bikelanes[forest_areas_with_bikelanes.geom_type.isin(['Polygon', 'MultiPolygon'])]
# forest_areas_with_bikelanes = forest_areas_with_bikelanes[forest_areas_with_bikelanes.geom_type.isin(['Polygon'])]

In [ ]:
print(set(forest_areas_with_bikelanes.geom_type))

In [ ]:
# forest_areas_with_bikelanes.drop(columns=['index_right0', 'index_right1', 'index_right2', 'h3_index'], inplace=True)
forest_areas_with_bikelanes.to_parquet('dataset/processed/forest_areas_with_bikelanes_wgs84_TEMP.parquet')

In [ ]:
forest_areas_with_bikelanes.sample(5000).explore()